<a href="https://colab.research.google.com/github/SonamTenzin7/FaceRecognition-using-facenet..../blob/main/FaceNet(Model_Trained).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow numpy matplotlib scikit-learn

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Images.zip to Images.zip


In [ ]:
import zipfile

# Path to the uploaded ZIP file
zip_path = list(uploaded.keys())[0]  # Get the name of the uploaded file

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('unzipped_files')  # Extract to a folder named 'unzipped_files'

print("Files unzipped successfully!")

Files unzipped successfully!


In [ ]:
import os
import numpy as np
from PIL import Image

def load_and_save_npy(image_dir, output_dir="."):
    images = []
    labels = []
    class_names = sorted(os.listdir(image_dir))

    for label, name in enumerate(class_names):
        class_path = os.path.join(image_dir, name)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = Image.open(img_path).convert('RGB')
            img = img.resize((160, 160))
            images.append(np.array(img))
            labels.append(label)

    # Convert to NumPy arrays
    X = np.stack(images).astype('float32') / 255.0  # Normalize to [0, 1]
    y = np.array(labels)

    # Save files
    np.save(os.path.join(output_dir, "images.npy"), X)
    np.save(os.path.join(output_dir, "labels.npy"), y)
    print(f"Saved {X.shape[0]} images (shape: {X.shape}) and labels.")

# Usage
load_and_save_npy("/content/unzipped_files/Images")

Saved 112 images (shape: (112, 160, 160, 3)) and labels.


# Load & Preprocess Dataset

In [ ]:
# Load dataset
X = np.load("images.npy")
y = np.load("labels.npy")

# Normalize images to [0,1] range
X = X.astype("float32") / 255.0

# Convert labels to integers
unique_labels = np.unique(y)
label_to_index = {label: i for i, label in enumerate(unique_labels)}
y = np.array([label_to_index[label] for label in y])

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)

Training data shape: (89, 160, 160, 3)
Validation data shape: (23, 160, 160, 3)


# Define FaceNet Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.models import Model

# Register the custom functions for serialization
@tf.keras.utils.register_keras_serializable()
def stack_embeddings(inputs):
    return tf.stack(inputs, axis=1)

@tf.keras.utils.register_keras_serializable()
def l2_normalize(x):
    return tf.math.l2_normalize(x, axis=1)

# Function to build the FaceNet model
def build_facenet(input_shape=(160, 160, 3), embedding_size=128):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(embedding_size)(x)

    x = Lambda(l2_normalize, name='l2_norm')(x)

    return Model(inputs, x, name="FaceNet")



# Implement the Triplet Loss


In [ ]:
def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[:, 0], y_pred[:, 1], y_pred[:, 2]

    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)

    loss = tf.maximum(pos_dist - neg_dist + alpha, 0.0)
    return tf.reduce_mean(loss)


# Build the Triplet Model

In [ ]:
from tensorflow.keras.layers import Lambda

def build_triplet_model(facenet):
    input_anchor = Input(shape=(160, 160, 3), name="anchor_input")
    input_positive = Input(shape=(160, 160, 3), name="positive_input")
    input_negative = Input(shape=(160, 160, 3), name="negative_input")

    # Generate embeddings
    anchor_embedding = facenet(input_anchor)
    positive_embedding = facenet(input_positive)
    negative_embedding = facenet(input_negative)

    # Stack embeddings using Lambda (Keras layer)
    def stack_embeddings(inputs):
        return tf.stack(inputs, axis=1)

    output = Lambda(stack_embeddings)([anchor_embedding, positive_embedding, negative_embedding])

    return Model([input_anchor, input_positive, input_negative], output, name="TripletNetwork")


# Create a Data Generator for Triplets

In [ ]:
import random
import numpy as np
import tensorflow as tf

def get_triplets(X, y, batch_size=16):
    def generator():
        while True:
            anchors, positives, negatives = [], [], []
            for _ in range(batch_size):
                # Randomly choose a class
                label = random.choice(np.unique(y))

                # Get indices of samples with that label
                anchor_idx, positive_idx = random.sample(list(np.where(y == label)[0]), 2)

                # Select a random negative sample from another class
                negative_label = random.choice([l for l in np.unique(y) if l != label])
                negative_idx = random.choice(np.where(y == negative_label)[0])

                # Append triplet samples as numpy arrays of float32
                anchors.append(X[anchor_idx].astype('float32'))
                positives.append(X[positive_idx].astype('float32'))
                negatives.append(X[negative_idx].astype('float32'))

            # Yield the triplet of samples
            yield (np.array(anchors), np.array(positives), np.array(negatives)), np.zeros((batch_size,))

    # Define output_signature to match dataset shape
    output_signature = (
        (tf.TensorSpec(shape=(batch_size, 160, 160, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(batch_size, 160, 160, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(batch_size, 160, 160, 3), dtype=tf.float32)),
        tf.TensorSpec(shape=(batch_size,), dtype=tf.float32)
    )

    return tf.data.Dataset.from_generator(generator, output_signature=output_signature)


In [ ]:
# Verify the first batch of triplets from the dataset
for batch in get_triplets(X_train, y_train, batch_size=16).take(1):
    print("Batch input shapes:", [b.shape for b in batch[0]])  # Check input shapes
    print("Batch label shape:", batch[1].shape)  # Should match batch size


Batch input shapes: [TensorShape([16, 160, 160, 3]), TensorShape([16, 160, 160, 3]), TensorShape([16, 160, 160, 3])]
Batch label shape: (16,)


In [ ]:
# Build the FaceNet model
face_embedding_model = build_facenet()

# Build the Triplet Model
triplet_model = build_triplet_model(face_embedding_model)


In [ ]:
print(triplet_model.summary())  # This confirms the model is correctly built


Model: "TripletNetwork"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ anchor_input (InputLayer) │ (None, 160, 160, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positive_input            │ (None, 160, 160, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ negative_input            │ (None, 160, 160, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ FaceNet (Functional)      │ (None, 128)            │     26,290,944 │ anchor_input[0][0],    │
│                           │                        │                │ positive_input[0][0],  │
│                           │                        │                │ negative_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_3 (Lambda)         │ (None, 3, 128)         │              0 │ FaceNet[0][0],         │
│                           │                        │                │ FaceNet[1][0],         │
│                           │                        │                │ FaceNet[2][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 26,290,944 (100.29 MB)

 Trainable params: 26,290,560 (100.29 MB)

 Non-trainable params: 384 (1.50 KB)

None


In [ ]:
# Compile the model with the triplet loss
triplet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=triplet_loss)


# Compile and Train the Model

In [ ]:
# Ensure your FaceNet model is built
face_embedding_model = build_facenet()

# Build the triplet model
triplet_model = build_triplet_model(face_embedding_model)

# Compile the model with your triplet loss function
triplet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=triplet_loss)

# Create datasets for training and validation
train_dataset = get_triplets(X_train, y_train, batch_size=16).repeat()
val_dataset = get_triplets(X_val, y_val, batch_size=16).repeat()

# Train the model
triplet_model.fit(
    train_dataset,
    validation_data=val_dataset,
    steps_per_epoch=len(X_train) // 16,
    validation_steps=len(X_val) // 16,
    epochs=20
)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 50s 9s/step - loss: 0.0232 - val_loss: 0.1849
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 82s 19s/step - loss: 0.0728 - val_loss: 0.1864
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 82s 18s/step - loss: 0.0213 - val_loss: 0.1854
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 43s 9s/step - loss: 0.0011 - val_loss: 0.1878
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 83s 19s/step - loss: 0.0019 - val_loss: 0.1877
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 81s 18s/step - loss: 0.0052 - val_loss: 0.1916
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 44s 9s/step - loss: 9.8428e-04 - val_loss: 0.1900
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 43s 9s/step - loss: 5.4364e-04 - val_loss: 0.1872
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 82s 18s/step - loss: 0.0000e+00 - val_loss: 0.1909
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 82s 18s/step - loss: 0.0000e+00 - val_loss: 0.1882
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 82s 18s/step - loss: 0.0000e+00 - val_loss: 0.1888
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 82s 18s/step - loss: 0.0000e+00 

# Save the Model

In [ ]:
# Save the entire model as an H5 file
triplet_model.save('triplet_face_model.h5')

# Evaluate the Model

In [ ]:
# Evaluate the model
validation_loss = triplet_model.evaluate(val_dataset, steps=len(X_val) // 16)
print(f"Validation Loss: {validation_loss}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.1779
Validation Loss: 0.17788799107074738


# Saving the model

In [ ]:
import os

# Make sure the directory exists
os.makedirs('/content/sample_data', exist_ok=True)

# Save model to a specific directory
triplet_model.save('/content/sample_data/triplet_face_model.h5')


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.models import Model

# Register stack_embeddings as a serializable function
@tf.keras.utils.register_keras_serializable()
def stack_embeddings(inputs):
    return tf.stack(inputs, axis=1)

@tf.keras.utils.register_keras_serializable()
def l2_normalize(x):
    return tf.math.l2_normalize(x, axis=1)

# Save the model after training
triplet_model.save('/content/sample_data/triplet_face_model.h5')


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

def cosine_similarity(embedding1, embedding2):
    """Compute cosine similarity between two embeddings."""
    return np.dot(embedding1, embedding2.T) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))


# Checking the model accuracy

In [ ]:
y_true = []
y_pred = []

for (img1, img2, label) in validation_pairs:
    # Convert images into embeddings using the face embedding model
    emb1 = face_embedding_model.predict(preprocess_image(img1))  # Get embedding for image 1
    emb2 = face_embedding_model.predict(preprocess_image(img2))  # Get embedding for image 2
A
    # Compute cosine similarity
    similarity = np.dot(emb1, emb2.T) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

    # Store results
    y_true.append(label)
    y_pred.append(1 if similarity > 0.5 else 0)  # Adjust threshold if needed

# Compute accuracy
accuracy = np.mean(np.array(y_true) == np.array(y_pred))
print(f"Face Recognition Accuracy: {accuracy * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
Face Recognition Accuracy: 50.00%
